In [20]:
import re

from langchain import hub
from langchain.agents.agent import AgentExecutor
from langchain.agents.react.agent import create_react_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.chat_models.base import ChatOpenAI

sql_prompt = hub.pull("langchain-ai/sql-agent-system-prompt")
react_prompt = hub.pull("hwchase17/react")
combined_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sql_prompt.messages[0].prompt.template + "\n" + react_prompt.template),
    ]
)
print(combined_prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'dialect', 'input', 'tool_names', 'tools', 'top_k'], input_types={}, partial_variables={}, template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a

In [25]:
sql_prompt.messages[0].prompt.template

'You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\n\nTo start you should ALWAYS look at the tables in the data

In [8]:
[
    *sql_prompt.messages,
]

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'top_k'], input_types={}, partial_variables={}, template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT m

In [16]:
x = sql_prompt.messages[0].prompt.template
print(x)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see 

In [18]:
x = react_prompt.template
print(x)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import re
from langchain.document_loaders import UnstructuredURLLoader
from langchain_core.documents.base import Document
from typing import List

url = "https://lilianweng.github.io/posts/2023-06-23-agent/"


def webloader(url: str) -> List[Document]:
    """Load the content of a website from url to text."""
    docs = WebBaseLoader(url).load()
    docs = [re.sub(r"\n{3,}", r"\n\n", doc.page_content) for doc in docs]
    return docs


print(webloader(url))

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import re
from langchain.document_loaders import UnstructuredURLLoader

url = "https://lilianweng.github.io/posts/2023-06-23-agent/"


print(UnstructuredURLLoader(urls=[url]).load())

In [ ]:
from docling.document_converter import DocumentConverter

source = "https://lilianweng.github.io/posts/2023-06-23-agent/"  # PDF path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())

In [1]:
from TextToSQL.react import text_to_sql_react
from TextToSQL.simple import text_to_sql


questions = [
    "What are the 10 most expensive albums",
]

for i, question in enumerate(questions, 1):
    print(f"\nQuestion {i}:\n{question}")
    # print(f"\nSimple Response {i}:")
    # response = text_to_sql(question)
    # print(response)
    print(f"\nReAct Response {i}:")
    response = text_to_sql_react(question)
    print(response)
    if i < len(questions):
        print("---")


Question 1:
What are the 10 most expensive albums

ReAct Response 1:
{'messages': [HumanMessage(content='What are the 10 most expensive albums', additional_kwargs={}, response_metadata={}, id='aba4e0b9-601f-4be6-afe0-7ea166906ec2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fKBLXJC4e7ASaczHOluSTG6m', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 554, 'total_tokens': 566, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-678c1b84-5645-40b6-9b41-db3310d05039-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_fKBLXJC4e7ASa

{'messages': [HumanMessage(content='What are the 10 most expensive albums', additional_kwargs={}, response_metadata={}, id='aba4e0b9-601f-4be6-afe0-7ea166906ec2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fKBLXJC4e7ASaczHOluSTG6m', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 554, 'total_tokens': 566, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-678c1b84-5645-40b6-9b41-db3310d05039-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_fKBLXJC4e7ASaczHOluSTG6m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 554, 'output_tokens': 12, 'total_tokens': 566, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content='Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', name='sql_db_list_tables', id='6c3e29b2-97fa-4867-baee-e94981f37084', tool_call_id='call_fKBLXJC4e7ASaczHOluSTG6m'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xYtQIfz0oOCQzkWp1tzzuTTw', 'function': {'arguments': '{"table_names":"Album"}', 'name': 'sql_db_schema'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 600, 'total_tokens': 616, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4bc80fe5-d059-4892-af8f-734ad8c441ae-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'Album'}, 'id': 'call_xYtQIfz0oOCQzkWp1tzzuTTw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 600, 'output_tokens': 16, 'total_tokens': 616, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content='\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/_\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n_/', name='sql_db_schema', id='1c917224-c194-4fe4-93d7-597021b42db5', tool_call_id='call_xYtQIfz0oOCQzkWp1tzzuTTw'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sR0ecMjp3YcN0qW8QHO9duey', 'function': {'arguments': '{"table_names":"InvoiceLine"}', 'name': 'sql_db_schema'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 738, 'total_tokens': 755, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-71b821d9-a82f-4a2e-8c58-9e16b64b570e-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'InvoiceLine'}, 'id': 'call_sR0ecMjp3YcN0qW8QHO9duey', 'type': 'tool_call'}], usage_metadata={'input_tokens': 738, 'output_tokens': 17, 'total_tokens': 755, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content='\nCREATE TABLE "InvoiceLine" (\n\t"InvoiceLineId" INTEGER NOT NULL, \n\t"InvoiceId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\t"Quantity" INTEGER NOT NULL, \n\tPRIMARY KEY ("InvoiceLineId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")\n)\n\n/_\n3 rows from InvoiceLine table:\nInvoiceLineId\tInvoiceId\tTrackId\tUnitPrice\tQuantity\n1\t1\t2\t0.99\t1\n2\t1\t4\t0.99\t1\n3\t2\t6\t0.99\t1\n_/', name='sql_db_schema', id='80466cf2-888c-4418-8ffb-f769a5ae672e', tool_call_id='call_sR0ecMjp3YcN0qW8QHO9duey'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cW5eOhHXtCRaHjiFy0mUaMh4', 'function': {'arguments': '{"table_names":"Track"}', 'name': 'sql_db_schema'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 932, 'total_tokens': 948, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6ca83738-1a13-4703-815d-f24ffd82a6d5-0', tool_calls=[{'name': 'sql_db_schema', 'args': {'table_names': 'Track'}, 'id': 'call_cW5eOhHXtCRaHjiFy0mUaMh4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 932, 'output_tokens': 16, 'total_tokens': 948, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content='\nCREATE TABLE "Track" (\n\t"TrackId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(200) NOT NULL, \n\t"AlbumId" INTEGER, \n\t"MediaTypeId" INTEGER NOT NULL, \n\t"GenreId" INTEGER, \n\t"Composer" NVARCHAR(220), \n\t"Milliseconds" INTEGER NOT NULL, \n\t"Bytes" INTEGER, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\tPRIMARY KEY ("TrackId"), \n\tFOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), \n\tFOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), \n\tFOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")\n)\n\n/_\n3 rows from Track table:\nTrackId\tName\tAlbumId\tMediaTypeId\tGenreId\tComposer\tMilliseconds\tBytes\tUnitPrice\n1\tFor Those About To Rock (We Salute You)\t1\t1\t1\tAngus Young, Malcolm Young, Brian Johnson\t343719\t11170334\t0.99\n2\tBalls to the Wall\t2\t2\t1\tU. Dirkschneider, W. Hoffmann, H. Frank, P. Baltes, S. Kaufmann, G. Hoffmann\t342562\t5510424\t0.99\n3\tFast As a Shark\t3\t2\t1\tF. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman\t230619\t3990994\t0.99\n_/', name='sql_db_schema', id='b54289fa-e4d9-47af-a6a9-93f685547674', tool_call_id='call_cW5eOhHXtCRaHjiFy0mUaMh4'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p3TiUJja4S3Q1vmmmMy7A6nI', 'function': {'arguments': '{"query":"SELECT a.Title, SUM(t.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN Track t ON a.AlbumId = t.AlbumId \\nJOIN InvoiceLine il ON t.TrackId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', 'name': 'sql_db_query'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 1289, 'total_tokens': 1376, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7eef5032-9319-4a8b-8778-13e7a6dd6bf8-0', tool_calls=[{'name': 'sql_db_query', 'args': {'query': 'SELECT a.Title, SUM(t.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN Track t ON a.AlbumId = t.AlbumId \nJOIN InvoiceLine il ON t.TrackId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;'}, 'id': 'call_p3TiUJja4S3Q1vmmmMy7A6nI', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1289, 'output_tokens': 87, 'total_tokens': 1376, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content="[('Battlestar Galactica (Classic), Season 1', 35.82), ('The Office, Season 3', 31.84), ('Minha Historia', 26.73), ('Heroes, Season 1', 25.87), ('Lost, Season 2', 25.87), ('Greatest Hits', 25.74), ('Unplugged', 24.75), ('Battlestar Galactica, Season 3', 23.88), ('Lost, Season 3', 21.89), ('Acústico', 21.78)]", name='sql_db_query', id='a272bc1f-bb51-4582-b142-8e4201edd0be', tool_call_id='call_p3TiUJja4S3Q1vmmmMy7A6nI'), AIMessage(content='The 10 most expensive albums based on total sales are:\n\n1. **Battlestar Galactica (Classic), Season 1** - $35.82\n2. **The Office, Season 3** - $31.84\n3. **Minha Historia** - $26.73\n4. **Heroes, Season 1** - $25.87\n5. **Lost, Season 2** - $25.87\n6. **Greatest Hits** - $25.74\n7. **Unplugged** - $24.75\n8. **Battlestar Galactica, Season 3** - $23.88\n9. **Lost, Season 3** - $21.89\n10. **Acústico** - $21.78', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 166, 'prompt_tokens': 1510, 'total_tokens': 1676, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1280}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-0064a075-8c2b-45ae-958e-9fea782bf249-0', usage_metadata={'input_tokens': 1510, 'output_tokens': 166, 'total_tokens': 1676, 'input_token_details': {'audio': 0, 'cache_read': 1280}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [27]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

response = {
    "messages": [
        HumanMessage(content="What are the 10 most expensive albums", additional_kwargs={}, response_metadata={}, id="08c293f3-c850-4861-bbae-ab333ea61dfc"),
        AIMessage(content="", additional_kwargs={"tool_calls": [{"id": "call_ujwhC8lXKlibq8mdGbL3hOQo", "function": {"arguments": "{}", "name": "sql_db_list_tables"}, "type": "function"}], "refusal": None}, response_metadata={"token_usage": {"completion_tokens": 12, "prompt_tokens": 554, "total_tokens": 566, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None}, id="run-00fb77df-ea09-45a7-9c04-075dc6692af4-0", tool_calls=[{"name": "sql_db_list_tables", "args": {}, "id": "call_ujwhC8lXKlibq8mdGbL3hOQo", "type": "tool_call"}], usage_metadata={"input_tokens": 554, "output_tokens": 12, "total_tokens": 566, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}}),
        ToolMessage(content="Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track", name="sql_db_list_tables", id="2639cd20-79be-4145-8c28-fd7c98c11998", tool_call_id="call_ujwhC8lXKlibq8mdGbL3hOQo"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_FkDZxE17O28rtluSfsePh1vy", "function": {"arguments": '{"table_names": "Album"}', "name": "sql_db_schema"}, "type": "function"}, {"id": "call_8br8wAw8mKZJrolWl8xnI3yn", "function": {"arguments": '{"table_names": "InvoiceLine"}', "name": "sql_db_schema"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 49, "prompt_tokens": 600, "total_tokens": 649, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-3b412371-8877-42c3-a5b3-0be49942be52-0",
            tool_calls=[{"name": "sql_db_schema", "args": {"table_names": "Album"}, "id": "call_FkDZxE17O28rtluSfsePh1vy", "type": "tool_call"}, {"name": "sql_db_schema", "args": {"table_names": "InvoiceLine"}, "id": "call_8br8wAw8mKZJrolWl8xnI3yn", "type": "tool_call"}],
            usage_metadata={"input_tokens": 600, "output_tokens": 49, "total_tokens": 649, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content='\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/', name="sql_db_schema", id="b4d20b87-a5d6-4fa7-b586-6eef045a68d4", tool_call_id="call_FkDZxE17O28rtluSfsePh1vy"),
        ToolMessage(content='\nCREATE TABLE "InvoiceLine" (\n\t"InvoiceLineId" INTEGER NOT NULL, \n\t"InvoiceId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\t"Quantity" INTEGER NOT NULL, \n\tPRIMARY KEY ("InvoiceLineId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")\n)\n\n/*\n3 rows from InvoiceLine table:\nInvoiceLineId\tInvoiceId\tTrackId\tUnitPrice\tQuantity\n1\t1\t2\t0.99\t1\n2\t1\t4\t0.99\t1\n3\t2\t6\t0.99\t1\n*/', name="sql_db_schema", id="44e0561a-fa3a-4454-a33d-ebd433e53ab8", tool_call_id="call_8br8wAw8mKZJrolWl8xnI3yn"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_T8fcCUSzvSrq1xkeiMnW1SAp", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 980, "total_tokens": 1054, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-084ef297-ca7b-41b8-9414-dff3295042fd-0",
            tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_T8fcCUSzvSrq1xkeiMnW1SAp", "type": "tool_call"}],
            usage_metadata={"input_tokens": 980, "output_tokens": 74, "total_tokens": 1054, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="728441e9-da64-4429-aaaa-283e8fad790f", tool_call_id="call_T8fcCUSzvSrq1xkeiMnW1SAp"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_An4AdvOZFahBfIs0bPr2ilEL", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1123, "total_tokens": 1197, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-8dd504f0-5099-45a5-85a9-d4e493b2c6be-0",
            tool_calls=[{"name": "sql_db_query", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_An4AdvOZFahBfIs0bPr2ilEL", "type": "tool_call"}],
            usage_metadata={"input_tokens": 1123, "output_tokens": 74, "total_tokens": 1197, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="Error: (sqlite3.OperationalError) no such column: il.AlbumId\n[SQL: SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)", name="sql_db_query", id="93c01a0d-4938-4e83-8923-622829d9f28c", tool_call_id="call_An4AdvOZFahBfIs0bPr2ilEL"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_3Zhdw8s6eUKZdqTSFJPdaday", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1304, "total_tokens": 1378, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1024}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-2587b203-5208-4361-93bd-b184ae9da2d4-0",
            tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_3Zhdw8s6eUKZdqTSFJPdaday", "type": "tool_call"}],
            usage_metadata={"input_tokens": 1304, "output_tokens": 74, "total_tokens": 1378, "input_token_details": {"audio": 0, "cache_read": 1024}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="e7a0ba29-540e-44b4-b15b-8a518db40a0e", tool_call_id="call_3Zhdw8s6eUKZdqTSFJPdaday"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_vFl1B4aUK7JT3Hiup2VuvYEE", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1447, "total_tokens": 1521, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1280}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-8636c183-a2e7-4655-9fba-0ebf41abfe4b-0",
            tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_vFl1B4aUK7JT3Hiup2VuvYEE", "type": "tool_call"}],
            usage_metadata={"input_tokens": 1447, "output_tokens": 74, "total_tokens": 1521, "input_token_details": {"audio": 0, "cache_read": 1280}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="da5403a8-0956-4342-93c1-346db7a38f8a", tool_call_id="call_vFl1B4aUK7JT3Hiup2VuvYEE"),
        AIMessage(
            content="",
            additional_kwargs={"tool_calls": [{"id": "call_Uyoj19aI1O4Zdrv705wIGYEg", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query"}, "type": "function"}], "refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 73, "prompt_tokens": 1590, "total_tokens": 1663, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1408}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
            id="run-eec068c0-f712-4c08-9fb3-1619b1eb6120-0",
            tool_calls=[{"name": "sql_db_query", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_Uyoj19aI1O4Zdrv705wIGYEg", "type": "tool_call"}],
            usage_metadata={"input_tokens": 1590, "output_tokens": 73, "total_tokens": 1663, "input_token_details": {"audio": 0, "cache_read": 1408}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
        ToolMessage(content="[('Balls to the Wall', 1.98), ('Warner 25 Anos', 1.98), ('Plays Metallica By Four Cellos', 1.98), ('The Best Of Buddy Guy - The Millenium Collection', 1.98), ('Carnaval 2001', 1.98), ('The Essential Miles Davis [Disc 1]', 1.98), ('The Battle Rages On', 1.98), ('Roda De Funk', 1.98), ('Demorou...', 1.98), ('Motley Crue Greatest Hits', 1.98)]", name="sql_db_query", id="70d28606-e327-48bb-927c-d48eff42b663", tool_call_id="call_Uyoj19aI1O4Zdrv705wIGYEg"),
        AIMessage(
            content="The 10 most expensive albums based on total sales are:\n\n1. **Balls to the Wall** - $1.98\n2. **Warner 25 Anos** - $1.98\n3. **Plays Metallica By Four Cellos** - $1.98\n4. **The Best Of Buddy Guy - The Millenium Collection** - $1.98\n5. **Carnaval 2001** - $1.98\n6. **The Essential Miles Davis [Disc 1]** - $1.98\n7. **The Battle Rages On** - $1.98\n8. **Roda De Funk** - $1.98\n9. **Demorou...** - $1.98\n10. **Motley Crue Greatest Hits** - $1.98\n\nAll these albums have a total price of $1.98 based on their sales.",
            additional_kwargs={"refusal": None},
            response_metadata={"token_usage": {"completion_tokens": 192, "prompt_tokens": 1803, "total_tokens": 1995, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1536}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "stop", "logprobs": None},
            id="run-c525515a-cb55-48f7-94b2-0746439137f2-0",
            usage_metadata={"input_tokens": 1803, "output_tokens": 192, "total_tokens": 1995, "input_token_details": {"audio": 0, "cache_read": 1536}, "output_token_details": {"audio": 0, "reasoning": 0}},
        ),
    ]
}

In [35]:
reversed_messages = reversed(response["messages"])
answer_msg = next((msg for msg in reversed_messages if isinstance(msg, AIMessage)), None)
answer = answer_msg.content if answer_msg else ""


data_msg = next((msg for msg in reversed_messages if isinstance(msg, ToolMessage) and msg.name == "sql_db_query"), None)
data = data_msg.content if data_msg else ""
query_call_id = data_msg.tool_call_id if data_msg else ""


query_msg = next((msg for msg in reversed_messages if isinstance(msg, AIMessage) and msg.tool_calls and any(call["id"] == query_call_id for call in msg.tool_calls)), None)
query = query_msg.tool_calls[0]["args"]["query"] if query_msg else ""

print(f"Answer: {answer}")
print(f"Data: {data}")
print(f"Query: {query}")

Answer: The 10 most expensive albums based on total sales are:

1. **Balls to the Wall** - $1.98
2. **Warner 25 Anos** - $1.98
3. **Plays Metallica By Four Cellos** - $1.98
4. **The Best Of Buddy Guy - The Millenium Collection** - $1.98
5. **Carnaval 2001** - $1.98
6. **The Essential Miles Davis [Disc 1]** - $1.98
7. **The Battle Rages On** - $1.98
8. **Roda De Funk** - $1.98
9. **Demorou...** - $1.98
10. **Motley Crue Greatest Hits** - $1.98

All these albums have a total price of $1.98 based on their sales.
Data: [('Balls to the Wall', 1.98), ('Warner 25 Anos', 1.98), ('Plays Metallica By Four Cellos', 1.98), ('The Best Of Buddy Guy - The Millenium Collection', 1.98), ('Carnaval 2001', 1.98), ('The Essential Miles Davis [Disc 1]', 1.98), ('The Battle Rages On', 1.98), ('Roda De Funk', 1.98), ('Demorou...', 1.98), ('Motley Crue Greatest Hits', 1.98)]
Query: SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice 
FROM Album a 
JOIN InvoiceLine il ON a.AlbumId = il.TrackId 
GROUP

In [ ]:
reversed_response = [
    AIMessage(
        content="The 10 most expensive albums based on total sales are:\n\n1. **Balls to the Wall** - $1.98\n2. **Warner 25 Anos** - $1.98\n3. **Plays Metallica By Four Cellos** - $1.98\n4. **The Best Of Buddy Guy - The Millenium Collection** - $1.98\n5. **Carnaval 2001** - $1.98\n6. **The Essential Miles Davis [Disc 1]** - $1.98\n7. **The Battle Rages On** - $1.98\n8. **Roda De Funk** - $1.98\n9. **Demorou...** - $1.98\n10. **Motley Crue Greatest Hits** - $1.98\n\nAll these albums have a total price of $1.98 based on their sales.",
        additional_kwargs={"refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 192, "prompt_tokens": 1803, "total_tokens": 1995, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1536}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "stop", "logprobs": None},
        id="run-c525515a-cb55-48f7-94b2-0746439137f2-0",
        usage_metadata={"input_tokens": 1803, "output_tokens": 192, "total_tokens": 1995, "input_token_details": {"audio": 0, "cache_read": 1536}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="[('Balls to the Wall', 1.98), ('Warner 25 Anos', 1.98), ('Plays Metallica By Four Cellos', 1.98), ('The Best Of Buddy Guy - The Millenium Collection', 1.98), ('Carnaval 2001', 1.98), ('The Essential Miles Davis [Disc 1]', 1.98), ('The Battle Rages On', 1.98), ('Roda De Funk', 1.98), ('Demorou...', 1.98), ('Motley Crue Greatest Hits', 1.98)]", name="sql_db_query", id="70d28606-e327-48bb-927c-d48eff42b663", tool_call_id="call_Uyoj19aI1O4Zdrv705wIGYEg"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_Uyoj19aI1O4Zdrv705wIGYEg", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 73, "prompt_tokens": 1590, "total_tokens": 1663, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1408}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-eec068c0-f712-4c08-9fb3-1619b1eb6120-0",
        tool_calls=[{"name": "sql_db_query", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_Uyoj19aI1O4Zdrv705wIGYEg", "type": "tool_call"}],
        usage_metadata={"input_tokens": 1590, "output_tokens": 73, "total_tokens": 1663, "input_token_details": {"audio": 0, "cache_read": 1408}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="da5403a8-0956-4342-93c1-346db7a38f8a", tool_call_id="call_vFl1B4aUK7JT3Hiup2VuvYEE"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_vFl1B4aUK7JT3Hiup2VuvYEE", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1447, "total_tokens": 1521, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1280}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-8636c183-a2e7-4655-9fba-0ebf41abfe4b-0",
        tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_vFl1B4aUK7JT3Hiup2VuvYEE", "type": "tool_call"}],
        usage_metadata={"input_tokens": 1447, "output_tokens": 74, "total_tokens": 1521, "input_token_details": {"audio": 0, "cache_read": 1280}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="e7a0ba29-540e-44b4-b15b-8a518db40a0e", tool_call_id="call_3Zhdw8s6eUKZdqTSFJPdaday"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_3Zhdw8s6eUKZdqTSFJPdaday", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1304, "total_tokens": 1378, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 1024}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-2587b203-5208-4361-93bd-b184ae9da2d4-0",
        tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_3Zhdw8s6eUKZdqTSFJPdaday", "type": "tool_call"}],
        usage_metadata={"input_tokens": 1304, "output_tokens": 74, "total_tokens": 1378, "input_token_details": {"audio": 0, "cache_read": 1024}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="Error: (sqlite3.OperationalError) no such column: il.AlbumId\n[SQL: SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)", name="sql_db_query", id="93c01a0d-4938-4e83-8923-622829d9f28c", tool_call_id="call_An4AdvOZFahBfIs0bPr2ilEL"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_An4AdvOZFahBfIs0bPr2ilEL", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 1123, "total_tokens": 1197, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-8dd504f0-5099-45a5-85a9-d4e493b2c6be-0",
        tool_calls=[{"name": "sql_db_query", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_An4AdvOZFahBfIs0bPr2ilEL", "type": "tool_call"}],
        usage_metadata={"input_tokens": 1123, "output_tokens": 74, "total_tokens": 1197, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="```sql\nSELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.AlbumId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;\n```", name="sql_db_query_checker", id="728441e9-da64-4429-aaaa-283e8fad790f", tool_call_id="call_T8fcCUSzvSrq1xkeiMnW1SAp"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_T8fcCUSzvSrq1xkeiMnW1SAp", "function": {"arguments": '{"query":"SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \\nFROM Album a \\nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \\nGROUP BY a.AlbumId, a.Title \\nORDER BY TotalPrice DESC \\nLIMIT 10;"}', "name": "sql_db_query_checker"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 74, "prompt_tokens": 980, "total_tokens": 1054, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-084ef297-ca7b-41b8-9414-dff3295042fd-0",
        tool_calls=[{"name": "sql_db_query_checker", "args": {"query": "SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice \nFROM Album a \nJOIN InvoiceLine il ON a.AlbumId = il.TrackId \nGROUP BY a.AlbumId, a.Title \nORDER BY TotalPrice DESC \nLIMIT 10;"}, "id": "call_T8fcCUSzvSrq1xkeiMnW1SAp", "type": "tool_call"}],
        usage_metadata={"input_tokens": 980, "output_tokens": 74, "total_tokens": 1054, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content='\nCREATE TABLE "InvoiceLine" (\n\t"InvoiceLineId" INTEGER NOT NULL, \n\t"InvoiceId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\t"Quantity" INTEGER NOT NULL, \n\tPRIMARY KEY ("InvoiceLineId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")\n)\n\n/*\n3 rows from InvoiceLine table:\nInvoiceLineId\tInvoiceId\tTrackId\tUnitPrice\tQuantity\n1\t1\t2\t0.99\t1\n2\t1\t4\t0.99\t1\n3\t2\t6\t0.99\t1\n*/', name="sql_db_schema", id="44e0561a-fa3a-4454-a33d-ebd433e53ab8", tool_call_id="call_8br8wAw8mKZJrolWl8xnI3yn"),
    ToolMessage(content='\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/', name="sql_db_schema", id="b4d20b87-a5d6-4fa7-b586-6eef045a68d4", tool_call_id="call_FkDZxE17O28rtluSfsePh1vy"),
    AIMessage(
        content="",
        additional_kwargs={"tool_calls": [{"id": "call_FkDZxE17O28rtluSfsePh1vy", "function": {"arguments": '{"table_names": "Album"}', "name": "sql_db_schema"}, "type": "function"}, {"id": "call_8br8wAw8mKZJrolWl8xnI3yn", "function": {"arguments": '{"table_names": "InvoiceLine"}', "name": "sql_db_schema"}, "type": "function"}], "refusal": None},
        response_metadata={"token_usage": {"completion_tokens": 49, "prompt_tokens": 600, "total_tokens": 649, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None},
        id="run-3b412371-8877-42c3-a5b3-0be49942be52-0",
        tool_calls=[{"name": "sql_db_schema", "args": {"table_names": "Album"}, "id": "call_FkDZxE17O28rtluSfsePh1vy", "type": "tool_call"}, {"name": "sql_db_schema", "args": {"table_names": "InvoiceLine"}, "id": "call_8br8wAw8mKZJrolWl8xnI3yn", "type": "tool_call"}],
        usage_metadata={"input_tokens": 600, "output_tokens": 49, "total_tokens": 649, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}},
    ),
    ToolMessage(content="Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track", name="sql_db_list_tables", id="2639cd20-79be-4145-8c28-fd7c98c11998", tool_call_id="call_ujwhC8lXKlibq8mdGbL3hOQo"),
    AIMessage(content="", additional_kwargs={"tool_calls": [{"id": "call_ujwhC8lXKlibq8mdGbL3hOQo", "function": {"arguments": "{}", "name": "sql_db_list_tables"}, "type": "function"}], "refusal": None}, response_metadata={"token_usage": {"completion_tokens": 12, "prompt_tokens": 554, "total_tokens": 566, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}, "model_name": "gpt-4o-mini-2024-07-18", "system_fingerprint": "fp_0ba0d124f1", "finish_reason": "tool_calls", "logprobs": None}, id="run-00fb77df-ea09-45a7-9c04-075dc6692af4-0", tool_calls=[{"name": "sql_db_list_tables", "args": {}, "id": "call_ujwhC8lXKlibq8mdGbL3hOQo", "type": "tool_call"}], usage_metadata={"input_tokens": 554, "output_tokens": 12, "total_tokens": 566, "input_token_details": {"audio": 0, "cache_read": 0}, "output_token_details": {"audio": 0, "reasoning": 0}}),
    HumanMessage(content="What are the 10 most expensive albums", additional_kwargs={}, response_metadata={}, id="08c293f3-c850-4861-bbae-ab333ea61dfc"),
]

In [17]:
answer = next((msg.content for msg in reversed_messages if isinstance(msg, int)), "")
print(answer)

In [5]:
response["messages"] = []

In [6]:
result = next(msg for msg in reversed(response["messages"]) if isinstance(msg, ToolMessage) and msg.name == "sql_db_query")
print(result.content)

StopIteration: 

In [14]:
query = next(msg for msg in reversed(response["messages"]) if isinstance(msg, ToolMessage) and msg.name == "sql_db_query_checker")
print(query.content)

```sql
SELECT a.Title, SUM(il.UnitPrice * il.Quantity) AS TotalPrice 
FROM Album a 
JOIN InvoiceLine il ON a.AlbumId = il.TrackId 
GROUP BY a.AlbumId, a.Title 
ORDER BY TotalPrice DESC 
LIMIT 10;
```
